<a href="https://colab.research.google.com/github/ulle9/TEST_REPO/blob/master/VKcup_docker1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install catboost

     |████████████████████████████████| 64.4MB 69kB/s 


In [5]:
import pandas as pd
import numpy as np
import sys
from catboost import CatBoostRegressor

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#DATA
# df_users = pd.read_csv(r'/content/drive/My Drive/vkcup/users.tsv', delimiter='\t')
df_history = pd.read_csv(r'/content/drive/My Drive/vkcup/history.tsv', delimiter='\t')
df_test = pd.read_csv(r'/content/drive/My Drive/vkcup/validate.tsv', delimiter='\t')
# df_asc_feat = pd.read_csv(r'/content/drive/My Drive/vkcup/acs_feat.csv', index_col=0)
# df_hist_feats = pd.read_csv(r'/content/drive/My Drive/vkcup/df_hist_feats.csv', index_col=0)

In [0]:
# def series_to_matrix(x, matrix):
#   n=0
#   for i in x:
#     for j in [int(k) for k in i.split(',')]:
#       # print(n,j-1)
#       matrix[n,j-1] = 1
#     n+=1

In [0]:
# # BIG MATRIX CREATION
# publishers = np.zeros([DF0.shape[0], 21]).astype(int)
# series_to_matrix(DF0.publishers, publishers)
# print(publishers.shape)
# users = np.zeros([DF0.shape[0], 27768]).astype(int)
# series_to_matrix(DF0.user_ids, users)
# print(users.shape)

# HISTORY FEATURE ENGINEERING

In [0]:
#DELTA HOUR
delta_hour = df_test.hour_end - df_test.hour_start
df_delta_hour = pd.DataFrame(delta_hour, columns = ['delta_hour']) 
# #audience_size/delta
# audsize_delta = (df_delta_hour.values.reshape(1,-1))/(DF0.audience_size.values.reshape(1,-1))
# df_audsize_delta = pd.DataFrame(audsize_delta.reshape(-1,1),  columns = ['audsize_delta']) 
# #cpm*delta
# cpm_delta = df_delta_hour.values.reshape(1,-1)*DF0.cpm.values.reshape(1,-1)
# df_cpm_delta = pd.DataFrame(cpm_delta.reshape(-1,1), columns = ['cpm_delta']) 

In [0]:
hist_feats = df_history[['cpm','hour','user_id']].groupby('user_id').agg({'cpm':'mean', 'hour':'count'}).\
  rename(columns={'cpm':'cpm_mean', 'hour':'show_counts'})

In [11]:
# HISTORY CPM & COUNT NEW
def func_hist_cpm(string):
  q = np.array([int(i) for i in string.split(',')])
  return hist_feats.loc[q,'cpm_mean'].sum()/len(q)
def func_hist_count(string):
  q = np.array([int(i) for i in string.split(',')])
  return hist_feats.loc[q,'show_counts'].sum()/len(q)

df_func_hist_cpm = df_test.user_ids.apply(func_hist_cpm)
df_func_hist_count = df_test.user_ids.apply(func_hist_count)
df_hist_feats = pd.DataFrame({'0':df_func_hist_cpm, '1': df_func_hist_count})

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


# SUMMARY

In [0]:
df_test1 = df_test.drop(['publishers', 'user_ids'], axis=1)
df_test1 = df_test1.drop(['hour_start', 'hour_end'], axis=1)
# DF1 = DF1.merge(df_asc_feat, left_index=True, right_index=True)
df_test1 = df_test1.merge(df_delta_hour, left_index=True, right_index=True)
df_test1 = df_test1.merge(df_hist_feats, left_index=True, right_index=True)
# DF1 = DF1.merge(df_audsize_delta, left_index=True, right_index=True)
# DF1 = DF1.merge(df_cpm_delta, left_index=True, right_index=True)

# PREDICTION

In [16]:
df_test1

,cpm,audience_size,delta_hour,0,1
0,220.0,1906,95,277.192969,40.567156
1,312.0,1380,6,179.039494,128.990580
2,70.0,888,20,281.236039,43.977477
3,240.0,440,82,253.803214,38.665909
4,262.0,1476,238,327.261172,35.524390
...,...,...,...,...,...
1003,127.0,368,59,251.821533,37.201087
1004,90.0,484,4,231.280577,41.793388
1005,122.0,704,5,178.998032,115.884943
1006,138.0,1210,237,250.687855,35.154545


In [150]:
# LOAD MODELS
ctr1 = CatBoostRegressor(early_stopping_rounds=10)
ctr2 = CatBoostRegressor(early_stopping_rounds=10)
ctr3 = CatBoostRegressor(early_stopping_rounds=10)
ctr1.load_model(r'/content/drive/My Drive/vkcup/ctr1')
ctr2.load_model(r'/content/drive/My Drive/vkcup/ctr2')
ctr3.load_model(r'/content/drive/My Drive/vkcup/ctr3')

In [0]:
# PREDICTION
pred10 = ctr1.predict(df_test1).clip(0.0000001,1)
pred20 = ctr2.predict(df_test1).clip(0.0000001,1)
pred30 = ctr3.predict(df_test1).clip(0.0000001,1)
prediction = pd.DataFrame({'at_least_one':pred10,'at_least_two':pred20, 'at_least_three':pred30})

In [0]:
# SAVE RESULT
prediction.to_csv(r'/content/drive/My Drive/vkcup/results.tsv', sep='\t', index=False, header=True)

# FIT MODELS BLOCK (comment before push!)

In [0]:
answers = pd.read_csv(r'/content/drive/My Drive/vkcup/validate_answers.tsv', delimiter='\t')

In [0]:
# CROSS-VALIDATION INDEXES
n = set(range(1008))
fold1_v = list(np.random.choice(list(n), 252, replace=False))
n -= set(fold1_v)
fold1_t = list(set(range(1008))-set(fold1_v))

fold2_v = list(np.random.choice(list(n), 252, replace=False))
n -= set(fold2_v)
fold2_t = list(set(range(1008))-set(fold2_v))

fold3_v = list(np.random.choice(list(n), 252, replace=False))
n -= set(fold3_v)
fold3_t = list(set(range(1008))-set(fold3_v))

fold4_v = list(n)
fold4_t = list(set(range(1008))-set(fold4_v))
folds = ((fold1_t, fold1_v), (fold2_t, fold2_v), (fold3_t, fold3_v), (fold4_t, fold4_v),)

In [163]:
# CROSS-VALIDATION SCORE
score = []
for i in folds:
  ctr1 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAPE')
  ctr2 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAPE')
  ctr3 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAPE')

  ctr1.fit(df_test1.iloc[i[0],:], answers.at_least_one.iloc[i[0]], eval_set=(df_test1.iloc[i[1],:], answers.at_least_one.iloc[i[1]]), verbose=False)
  ctr2.fit(df_test1.iloc[i[0],:], answers.at_least_two.iloc[i[0]], eval_set=(df_test1.iloc[i[1],:], answers.at_least_two.iloc[i[1]]), verbose=False)
  ctr3.fit(df_test1.iloc[i[0],:], answers.at_least_three.iloc[i[0]], eval_set=(df_test1.iloc[i[1],:], answers.at_least_three.iloc[i[1]]), verbose=False)

  pred1 = ctr1.predict(df_test1).clip(0.0000001,1)
  pred2 = ctr2.predict(df_test1).clip(0.0000001,1)
  pred3 = ctr3.predict(df_test1).clip(0.0000001,1)
  predictions = pd.DataFrame({'at_least_one':pred1,'at_least_two':pred2, 'at_least_three':pred3})
  print(ctr1.best_score_, ctr1.tree_count_)
  print(get_smoothed_mean_log_accuracy_ratio(answers, predictions))
  score.append(get_smoothed_mean_log_accuracy_ratio(answers, predictions))
print(score, sum(score)/4)

{'learn': {'MAPE': 0.057318940133533176}, 'validation': {'MAPE': 0.07305244917681081}} 120
59.79
{'learn': {'MAPE': 0.060942006906858186}, 'validation': {'MAPE': 0.07355019320384334}} 73
64.23
{'learn': {'MAPE': 0.059147408346681425}, 'validation': {'MAPE': 0.07289210186517844}} 102
62.15
{'learn': {'MAPE': 0.06426590194473146}, 'validation': {'MAPE': 0.06081446265408669}} 73
71.94
[59.79, 64.23, 62.15, 71.94] 64.5275


In [167]:
# FULL_DF FIT/PREDICT
ctr1 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAPE')
ctr2 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAPE')
ctr3 = CatBoostRegressor(early_stopping_rounds=10, loss_function='MAPE')
ctr1.fit(df_test1, answers.at_least_one, verbose=False)
ctr2.fit(df_test1, answers.at_least_two, verbose=False)
ctr3.fit(df_test1, answers.at_least_three, verbose=False)
pred1 = ctr1.predict(df_test1).clip(0.0000001,1)
pred2 = ctr2.predict(df_test1).clip(0.0000001,1)
pred3 = ctr3.predict(df_test1).clip(0.0000001,1)
prediction = pd.DataFrame({'at_least_one':pred1,'at_least_two':pred2, 'at_least_three':pred3})
print(get_smoothed_mean_log_accuracy_ratio(answers, prediction))

36.36


In [0]:
# SAVE MODELS
ctr1.save_model(r'/content/drive/My Drive/vkcup/ctr1')
ctr2.save_model(r'/content/drive/My Drive/vkcup/ctr2')
ctr3.save_model(r'/content/drive/My Drive/vkcup/ctr3')

In [122]:
a = ctr1.get_params()
a


{'early_stopping_rounds': 10, 'loss_function': 'RMSE'}

In [0]:
# METRIC
def get_smoothed_log_mape_column_value(responses_column, answers_column, epsilon):
    return np.abs(np.log((responses_column + epsilon)/(answers_column + epsilon) )).mean()


def get_smoothed_mean_log_accuracy_ratio(answers, responses, epsilon=0.005):
    log_accuracy_ratio_mean = np.array(
        [   get_smoothed_log_mape_column_value(responses.at_least_one, answers.at_least_one, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_two, answers.at_least_two, epsilon),
            get_smoothed_log_mape_column_value(responses.at_least_three, answers.at_least_three, epsilon)]).mean()

    percentage_error = 100 * (np.exp(log_accuracy_ratio_mean) - 1)
    return percentage_error.round(decimals=2)
print(get_smoothed_mean_log_accuracy_ratio(answers, prediction))

In [152]:
print(get_smoothed_mean_log_accuracy_ratio(answers, prediction))

41.5
